# SQL UNION

###  Содержание <a class="anchor" id=0></a>
- [1. Знакомимся с данными](#1)
- [2. UNION](#2)
- [2.1 Виды UNION](#2-1)
- [3. UNION и ограничение типов данных](#3)
- [4. UNION ALL и промежуточные итоги](#4)
- [5. UNION и дополнительные условия](#5)
- [6. UNION и ручная генерация](#6)
- [7. EXCEPT](#6)
- [8. INTERSECT](#6)
- [9. Итоги. Закрепление знаний](#6)
- [10. Дополнительные задачи по SQL](#6)


## Знакомимся с данными <a class="anchor" id=1></a>

[к содержанию](#0)

В данном блоке мы будем работать с данными о компании, организующей перевозки грузов.

Интересующие нас данные хранятся в таблицах `city`, `customer`, `driver`, `shipment`, `truck`. Давайте внимательно их изучим.

Ниже представлена `ER`-диаграмма (от англ. `entity-relation`, дословно — «сущность-связь»), которая отображает существующие связи между отдельными таблицами.

<img src=sql_4_img1.jpg>

Таблица `city` — это справочник городов. Структура справочника представлена ниже.

<img src=sql_4_img2.png>

Таблица `customer` — это справочник клиентов. У компании, с данными которой мы работаем, только корпоративные клиенты, поэтому в таблице нет привычных данных о возрасте и поле. Справочник содержит следующие поля:

<img src=sql_4_img3.png>

Следующая таблица — `driver` — справочник водителей. Перечень сведений, содержащихся в таблице, представлен ниже.

<img src=sql_4_img4.png>

В таблице `truck` хранится информация о грузовиках, на которых осуществляются перевозки. Данные о них представлены в следующем виде:

<img src=sql_4_img5.png>

Последняя таблица в датасете, `shipment`, — таблица с данными непосредственно о доставках. Она описывает взаимодействие всех перечисленных сущностей, а потому содержит наибольшее количество ссылок на другие таблицы.

<img src=sql_4_img6.png>

In [ ]:
/* Укажите название города с максимальным весом единичной доставки. */
select
    c.city_name,
    s.weight
from
    sql.city c
join sql.shipment s on s.city_id = c.city_id
order by 2 desc

/* Сколько различных производителей грузовиков перечислено в таблице truck? */
select
    count(distinct make)
from
    sql.truck

/* Как зовут водителя (first_name), который совершил наибольшее количество доставок одному клиенту? */
select
    d.first_name,
    count(distinct(s.cust_id))
from
    sql.driver d 
join sql.shipment s on d.driver_id = s.driver_id
group by d.first_name
order by 2 desc

/* Укажите даты первой и последней по времени доставок в таблице shipment. */
select
    min(ship_date),
    max(ship_date)
from
    sql.shipment

/* Укажите имя клиента, получившего наибольшее количество доставок за 2017 год. */
select
    cus.cust_name,
    count(s.ship_id),
    extract(year from s.ship_date)
from
    sql.customer cus
join sql.shipment s on cus.cust_id = s.cust_id
group by 1,3
having extract(year from s.ship_date) = 2017
order by 2 desc


## UNION <a class="anchor" id=2></a>

[к содержанию](#0)

Допустим, мы хотим собрать из справочников по книгам и фильмам один, так чтобы в нём содержались названия произведений, а также их описание — книга или фильм.

Для этого напишем простой запрос:

In [ ]:
SELECT book_name object_name, 'книга' object_descritption /*выбираем столбец с названием book_name, задаём алиас для столбца object_name, задаём во второй колонке объект ‘книга’ с алиасом для столбца object_descritption*/
FROM public.books /*из схемы public и таблицы books*/
UNION ALL /*оператор присоединения*/
SELECT movie_title, 'фильм' /*выбираем столбец movie_title, сами задаём во второй колонке объект ‘фильм’*/
FROM sql.kinopoisk /*из схемы sql и таблицы kinopoisk*/

Визуально произведённое нами действие можно представить следующим образом:

<img src=sql_4_img7.png>

Общий принцип мы поняли, разберёмся в деталях:

В запросе мы использовали оператор `UNION ALL` — он присоединяет любой результат запроса к другому «снизу» при условии, что у них одинаковая структура, а именно:

#### одинаковый тип данных

<img src=sql_4_img8.png>

#### одинаковое количество столбцов

<img src=sql_4_img9.png>

#### одинаковый порядок столбцов согласно типу данных

<img src=sql_4_img10.png>

## ВИДЫ UNION <a class="anchor" id=2-1></a>

[к содержанию](#0)

Оператор присоединения существует в двух вариантах:

* `UNION` выводит только **уникальные записи**;

* `UNION ALL` присоединяет **все строки** последующих таблиц к предыдущим, без ограничений по уникальности.

>**Важно!** `UNION` оставляет только уникальные значения, а потому требует дополнительных вычислительных мощностей и памяти (в данном случае можно провести аналогию с `DISTINCT`). Поэтому если вы уверены в отсутствии дубликатов в данных или они вам не важны, предпочтительнее использовать `UNION ALL`.

### СИНТАКСИС

Запрос строится таким образом:

In [ ]:
SELECT
    n columns
FROM 
    table_1

UNION ALL

SELECT 
    n columns
FROM 
    table_2
...

UNION ALL

SELECT 
    n columns
FROM 
    table_n

Результатом выполнения такого запроса будут строки `table_1`, `table_2`, `...`, `table_n`, соединённые одни под другими и выведенные в единой выдаче.

>**Важно!** Названия итоговых колонок в выводе будут такие же, как в первом блоке `SELECT`, даже если они отличаются в других блоках подзапросов.
Пришла пора испытать функцию `UNION(ALL)` на практике.

Обратимся к нашему датасету о транспортной компании и посмотрим, как сформировать справочник с `ID` всех таблиц и указанием объекта, к которому он относится

In [ ]:
SELECT
    c.city_id object_name, 'id города' object_type
FROM 
    sql.city c
UNION ALL
SELECT
    d.driver_id other_name, 'id водителя' other_type
FROM 
    sql.driver d
UNION ALL
SELECT
    s.ship_id, 'id доставки'
FROM 
    sql.shipment s
UNION ALL
SELECT
    c.cust_id, 'id клиента'
FROM 
    sql.customer c
UNION ALL
SELECT
    t.truck_id, 'id грузовика'
FROM 
    sql.truck t
ORDER BY 1

>**Обратите внимание!** Несмотря на исходные названия колонок `other_name` и `other_type` во втором подзапросе, в выводе мы получим названия, которые дали в первом блоке: `object_name` и `object_type`.

Другая особенность — в применении сортировки `ORDER BY`: она всегда будет относиться **к итоговому результату** всего запроса с `UNION ALL`.

В случаях, когда необходимо применить команду `ORDER BY` или `LIMIT` не к итоговому результату, а к каждой части запроса, можно **обернуть подзапросы в скобки**.

Чтобы посмотреть, как это работает, вернёмся к нашему примеру с общим справочником по фильмам и книгам.

Мы уже знаем, что можно легко и непринуждённо применить операторы `ORDER BY` и `LIMIT` ко всему результату запроса.

In [ ]:
SELECT book_name object_name, 'книга' object_descritption 
FROM public.books
UNION ALL
SELECT movie_title, 'фильм' 
FROM sql.kinopoisk
ORDER BY 1
LIMIT 1

Всё бы хорошо, только в таком случае отсортирован будет весь общий справочник, а в выводе останется одна строка с названием объекта, идущим первым по алфавиту.

А если мы не хотим общую сортировку? Может, нам нужны строки с названием как фильма, так и книги, идущих первыми по алфавиту.

Нет ничего проще — отсортируем каждую часть запроса **по отдельности и объединим результаты!**

In [ ]:
(SELECT book_name object_name, 'книга' object_descritption 
FROM public.books
ORDER BY 1
LIMIT 1)
UNION ALL
(SELECT movie_title, 'фильм' 
FROM sql.kinopoisk
ORDER BY 1
LIMIT 1)

Напишите запрос, который создаёт уникальный алфавитный справочник всех городов, штатов, имён водителей и производителей грузовиков.
Результатом запроса должны быть два столбца: название и тип объекта (city, state, driver, truck).
Отсортируйте список по названию объекта, а затем — по типу.

In [ ]:
SELECT c.city_name "название", 'city' "тип объекта"
FROM sql.city c
UNION
SELECT c.state, 'state'
FROM sql.city c
UNION
SELECT d.first_name, 'driver'
FROM sql.driver d
UNION
SELECT t.make, 'truck'
FROM sql.truck t
ORDER BY 1,2


Напишите запрос, который соберёт имена всех упомянутых городов и штатов с таблицы `city`.
Результатом запроса должен быть один столбец `object_name`, отсортированный в алфавитном порядке.

In [ ]:
SELECT
    c.city_name object_name
FROM 
    sql.city c
UNION ALL
SELECT
    c.state
FROM 
    sql.city c
ORDER BY 1

Выполнив предыдущий запрос, мы получили города с одинаковыми названиями, но находящиеся в разных штатах, а также большое количество дублирующихся названий штатов.
Перепишите предыдущий запрос так, чтобы остались только уникальные названия городов и штатов.
Результатом запроса должен быть один столбец object_name, отсортированный в алфавитном порядке.

In [ ]:
SELECT
    c.city_name object_name
FROM 
    sql.city c
UNION
SELECT
    c.state
FROM 
    sql.city c
ORDER BY 1

## UNION и ограничение типов данных <a class="anchor" id=3></a>

[к содержанию](#0)

### ПОЧЕМУ ТАК ВАЖЕН ТИП ДАННЫХ?

Как мы уже знаем, `UNION` может быть использован только в случае полного соответствия столбцов и их типов в объединяемых запросах.

Допустим, мы хотим вывести список всех `id` городов и их названий в одном столбце.

In [ ]:
/* ебанем в лоб */
SELECT 
    c.city_id
FROM
    sql.city c
UNION ALL
SELECT 
    cc.city_name
FROM
    sql.city cc


**"ERROR: UNION types integer and text cannot be matched"**

>Если мы всё же хотим выполнить поставленную задачу, придётся привести оба столбца к одному типу данных. Не каждый текст может быть приведён к числу, зато каждое число может быть представлено в текстовом формате.

Забегая вперёд, скажем пару слов о типизации столбцов. Для типизации в Postgres составляется запрос по модели `column_name::column_type`.

Таким образом, чтобы перевести `city_id` в текст, нам потребуется написать `city_id::text`.

>**Важно!** Любой тип данных может быть приведён к текстовому формату — эту возможность целесообразно использовать для соединения разнородных сущностей. Главное — помнить, что сортировка текста отличается от сортировки чисел и дат.

In [ ]:
SELECT 
    c.city_id::text
FROM
    sql.city c
UNION ALL
SELECT 
    cc.city_name
FROM
    sql.city cc

Напишите запрос, который объединит в себе все почтовые индексы водителей и их телефоны в единый столбец-справочник. Также добавьте столбец с именем водителя и столбец с типом контакта (`phone` или `zip` в зависимости от типа). Столбцы к выводу: `contact`, `first_name`, `contact_type`.
Отсортируйте список по столбцу с контактными данными в порядке возрастания, а затем — по имени водителя.

In [ ]:
SELECT
    d.phone "contact",
    d.first_name,
    'phone' "contact_type"
FROM sql.driver d
union all
SELECT
    d.zip_code::text "contact",
    d.first_name,
    'zip' "contact_type"
FROM sql.driver d
ORDER BY 1, 2

## UNION ALL и промежуточные итоги<a class="anchor" id=4></a>

[к содержанию](#0)

## UNION и дополнительные условия <a class="anchor" id=5></a>

[к содержанию](#0)

## UNION и ручная генерация <a class="anchor" id=6></a>

[к содержанию](#0)

## EXCEPT <a class="anchor" id=7></a>

[к содержанию](#0)

## INTERSECT <a class="anchor" id=8></a>

[к содержанию](#0)

## Итоги. Закрепление знаний <a class="anchor" id=9></a>

[к содержанию](#0)

## Дополнительные задачи по SQL <a class="anchor" id=10></a>

[к содержанию](#0)